In [1]:
import numpy as np
import pandas as pd
import vectorbt as vbt
import sys
import os
function_essential_path = r"H:\Desktop\Environement_Trading_Developement\FunctionEssential"
sys.path.append(function_essential_path)
import utils as us
import talib 

In [2]:
def point_pivot(close, high, low, nbr_pivot):
    pivot_collection = []
    pivot_support_0 = (close + high + low) / 3
    pivot_collection.append(pivot_support_0)
    pivot_resistance_0 = (close + high + low) / 3
    pivot_collection.append(pivot_resistance_0)
    pivot_support_1 = 2 * pivot_support_0 - high
    pivot_resistance_1 = 2 * pivot_support_0 - low
    pivot_collection.append(pivot_support_1)
    pivot_collection.append(pivot_resistance_1)
    for i in range(2, nbr_pivot):
        pivot_support_i = pivot_support_0 - (high - low) * (i - 1)
        pivot_resistance_i = pivot_support_0 + (high - low) * (i - 1)
        pivot_collection.append(pivot_support_i)
        pivot_collection.append(pivot_resistance_i)
    point_pivot_array = np.array(pivot_collection).T
    point_pivot_array = np.squeeze(point_pivot_array)
    return point_pivot_array

In [3]:
def entries(ma300,ma200, ma100, ma50, ma15,adx, close):
    # Vérification des conditions initiales
    condition_initiale_long = (
        (ma300 > ma200) &
        (ma200 > ma100) &
        (ma100 > ma50) &
        (ma50 > ma15) &
        (adx > 25)
    )
    
    # Vérification de la persistance 5 bougies plus tard
    condition_persistante = condition_initiale_long.shift(5).fillna(False)
    
    # Signal d'entrée quand les deux conditions sont remplies
    entries_long = condition_initiale_long & condition_persistante

    condition_initiale_short = (
        (ma300 < ma200) &
        (ma200 < ma100) &
        (ma100 < ma50) &
        (ma50 < ma15) &
        (adx > 25)
    )

    condition_persistante_short = condition_initiale_short.shift(5).fillna(False)
    
    # Signal d'entrée quand les deux conditions sont remplies
    entries_short = condition_initiale_short & condition_persistante_short
    
    return entries_long,entries_short

def exits(close,high,low, long, short, point_pivot_df):

    exits_long = [False] * len(long)
    exits_short = [False] * len(short)

    for i in range(len(close)):
        if long[i]:
            # Utilisation de .iloc pour l'accès positionnel
            SL = point_pivot_df['pivot_support_2'].iloc[i]
            TP = point_pivot_df['pivot_resistance_3'].iloc[i]
            
            # Vérification des valeurs NaN
            if pd.isna(SL) or pd.isna(TP):
                continue
                
            for j in range(i + 1, len(close)):
                if low[j] <= SL or high[j] >= TP:
                    exits_long[j] = True
                    break  # Sortie après la première occurrence

        if short[i]:
            SL = point_pivot_df['pivot_resistance_2'].iloc[i]
            TP = point_pivot_df['pivot_support_3'].iloc[i]
            
            if pd.isna(SL) or pd.isna(TP):
                continue
                
            for j in range(i + 1, len(close)):
                if high[j] >= SL or low[j] <= TP:
                    exits_short[j] = True
                    break

    return exits_long, exits_short

In [4]:
class Strategie():
    

    def __init__(self,data,frequence,list_tickers):
        self.data = data
        self.frequence = frequence
        self.list_tickers = list_tickers


    def backtest(self,period_start,period_end):
        if (period_end == 0) and (period_start == 0):

            entries_long_df = pd.DataFrame()
            entries_short_df = pd.DataFrame()
            exits_long_df = pd.DataFrame()
            exits_short_df = pd.DataFrame()
            close_df = pd.DataFrame()

            for ticker in self.list_tickers:
                data = self.data[ticker]
                liste_value = self.make_value_no_combi(data['close'],data['high'],data['low'],data.index)
                long,short = self.make_entries(liste_value,data['close'])
                exits_long,exits_short = self.make_exits(data['close'],data['high'],data['low'],long,short,liste_value[6])



                exits_long = pd.DataFrame(exits_long)
                exits_short = pd.DataFrame(exits_short)
                #Reset les index pour tout normaliser
                long = long.reset_index(drop=True)
                short = short.reset_index(drop=True)
                exits_long = exits_long.reset_index(drop=True)
                exits_short = exits_short.reset_index(drop=True)
                close = data['close'].reset_index(drop=True)
                #enregistrer dans des dataframes 
                entries_long_df[ticker] = long
                entries_short_df[ticker] = short
                exits_long_df[ticker] = exits_long
                exits_short_df[ticker] = exits_short
                close_df[ticker] = close

            #remplacer les 0 qui ont état mis durant import data
            close_df = close_df.replace(0, None).fillna(method='ffill')
            portfolio = vbt.Portfolio.from_signals(
                close_df,
                entries_long_df,
                exits_long_df,
                entries_short_df,
                exits_short_df,
                freq=self.frequence
            )
            us.generate_portfolio_report(portfolio,close_df)
            us.get_pnl(portfolio)
            return portfolio
        
        else :
            pass


    def optimize(self,metrics,choice,period1,period2,params1):
        pass

    
    def make_entries(self,liste_value,close):
        long,short = entries(liste_value[0],liste_value[1],liste_value[2],liste_value[3],liste_value[4],liste_value[5],close)
        return long,short

    def make_exits(self,close,high,low,long,short,point_pivot_df):
        exits_long,exits_short = exits(close,high,low,long,short,point_pivot_df)
        return exits_long,exits_short

    
    def make_value_no_combi(self,close,high,low,datetime):
        adx = talib.ADX(high,low,close,timeperiod=14)
        print(adx.shape)
        ma300 = vbt.MA.run(close,300).ma
        ma200 = vbt.MA.run(close,200).ma
        ma100 = vbt.MA.run(close,100).ma
        ma50 = vbt.MA.run(close,50).ma
        ma15 = vbt.MA.run(close,15).ma
        ma10 = vbt.MA.run(close,10).ma

        data_weekly = us.get_data_indice(['FDAX_TOTAL_4H'],'W1')
        data_weekly = data_weekly['FDAX_TOTAL_4H']
        point_pivot_value = point_pivot(data_weekly['close'],data_weekly['high'],data_weekly['low'],4)
        columns = []
        count = 0
        for i in range(point_pivot_value.shape[1]):
            if i % 2 == 0:
                columns.append(f"pivot_support_{count}") 
            else:
                columns.append(f"pivot_resistance_{count}") 
                count += 1
        point_pivot_df = pd.DataFrame(point_pivot_value,columns=columns,index=data_weekly.index)
        point_pivot_df.index = pd.to_datetime(point_pivot_df.index)
        datetime = pd.to_datetime(datetime)
        point_pivot_df = point_pivot_df.reindex(datetime, method='ffill')
        print(point_pivot_df.shape)
        print(ma200.shape)

        return [ma300,ma200,ma100,ma50,ma15,adx,point_pivot_df]
        



In [5]:
tickers=['FDAX_TOTAL_4H']
data = us.get_data_indice(tickers,'D1')

In [6]:
strat = Strategie(data,'240m',tickers)
strat.backtest(0,0)



(6542,)
(6542, 8)
(6542,)


C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_4760\723025166.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  condition_persistante = condition_initiale_long.shift(5).fillna(False)
C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_4760\723025166.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  condition_persistante_short = condition_initiale_short.shift(5).fillna(False)
C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_4760\723025166.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future versio


-----------------------REPORT METRICS-----------------------


Total Return : -55.43%
Benchmark return : 163.48%
CAGR : -23.7%
Volatility : 37.57%
Sharpe Ratio : -0.53
Benchmark sharpe ratio : 0.33
Max Drawdown: -61.86%
Calmar Ratio : -0.38
Beta : 0.23


--------------- REPORT METRICS TRADES--------------------


Total trades : 35
Total long trades 11
Total short trades 24
Win rate : 0.2571428571428571%
Wining streak : 2.0
Loosing streak : 9.0
Average winning trade : 7.000000000000001%
Average losing trade : -5.0%
Profit factor : 0.35
Expectancy : -1.58%

-----------------------------------------------------------

